# **Tiny Imagenet - Super Resolution**

We will be working on super-resolution and not classification tasks in this NB.

In [ ]:
import os, timm, torch, random, datasets, math, fastcore.all as fc 
import numpy as np, matplotlib as mpl, matplotlib.pyplot as plt
import torchvision.transforms as T
import torchvision.transforms.functional as TF,torch.nn.functional as F

from torch.utils.data import DataLoader,default_collate
from pathlib import Path
from torch.nn import init
from fastcore.foundation import L
from torch import nn,tensor
from datasets import load_dataset
from operator import itemgetter
from torcheval.metrics import MulticlassAccuracy
from functools import partial
from torch.optim import lr_scheduler
from torch import optim
from torchvision.io import read_image,ImageReadMode

from miniai.datasets import *
from miniai.conv import *
from miniai.learner import *
from miniai.activations import *
from miniai.init import *
from miniai.sgd import *
from miniai.resnet import *
from miniai.augment import *
from miniai.accel import *
from miniai.training import *

In [ ]:
from fastprogress import progress_bar
from glob import glob
import warnings; warnings.filterwarnings('ignore')

In [ ]:
torch.set_printoptions(precision=5, linewidth=140, sci_mode=False)
torch.manual_seed(1)
mpl.rcParams['figure.dpi'] = 70

set_seed(42)
if fc.defaults.cpus>8: fc.defaults.cpus=8

## **Data Processing**

Copying over sections from the previous NBs.

In [ ]:
path_data = Path('data')
path = path_data/'tiny-imagenet-200'
bs = 512
xmean,xstd = (tensor([0.47565, 0.40303, 0.31555]), tensor([0.28858, 0.24402, 0.26615]))

In [ ]:
path

**For tasks like super-resolution (and image reconstruction in general) we must ensure that both the dependant and independant data receive the same amount of data augmentation.** 

In [ ]:
tfms = nn.Sequential(T.Pad(8), T.RandomCrop(64), T.RandomHorizontalFlip())

In [ ]:
class TinyDS:
    def __init__(self, path):
        self.path = Path(path)
        self.files = glob(str(path/'**/*.JPEG'), recursive=True)
    def __len__(self): return len(self.files)
    def __getitem__(self, i):
        img = read_image(self.files[i], mode=ImageReadMode.RGB)/255
        return tfms((img-xmean[:,None,None])/xstd[:,None,None])

class TfmDS:
    def __init__(self, ds, tfmx=fc.noop, tfmy=fc.noop): self.ds,self.tfmx,self.tfmy = ds,tfmx,tfmy
    def __len__(self): return len(self.ds)
    def __getitem__(self, i):
        item = self.ds[i]
        return self.tfmx(item),self.tfmy(item)

def denorm(x): return (x*xstd[:,None,None]+xmean[:,None,None]).clamp(0,1)

- We will add random erasing just for the training set. This makes the task of learning more difficult for the model and forces it to learn to replace missing pixels.
- In `TfmDS` the argument `tfmx` applies transformations / augmentations to the independant variable. In turn `tfmx()` applies `TF.resize()` to the 32x32 image, making it 64x64 and interpolates. This effectively doubles up the pixel count. 

In [ ]:
# We will add random erasing just for the training set. This makes the task of learning more
# difficult for the model and forces it to learn to replace missing pixels.
def tfmx(x, erase=True):
    x = TF.resize(x, (32,32))[None]
    x = F.interpolate(x, scale_factor=2)
    if erase: x = rand_erase(x)
    return x[0]

In [ ]:
tds = TinyDS(path/'train')
vds = TinyDS(path/'val')

tfm_tds = TfmDS(tds, tfmx)
tfm_vds = TfmDS(vds, partial(tfmx, erase=False))

dls = DataLoaders(*get_dls(tfm_tds, tfm_vds, bs=bs, num_workers=8))

In [ ]:
xb,yb = next(iter(dls.train))

In [ ]:
show_images(denorm(xb[:4]), imsize=2.5)

In [ ]:
show_images(denorm(yb[:4]), imsize=2.5)

## **Implementing a Denoising AutoEncoder**

The `upblock` basically doubles the pixels through sequential upsampling. This is followed by an extra Resblock at the end to return 3 channels.

So this model basically takes images which are squeezed into smaller representations, and then these representations are brought back up to their super-resolution target. 

In [ ]:
def upblock(ni, nf, ks=3, act=act_gr, norm=None):
    return nn.Sequential(nn.UpsamplingNearest2d(scale_factor=2),
                         ResBlock(ni, nf, ks=ks, act=act, norm=norm))

In [ ]:
def get_model(act=act_gr, nfs=(32, 64, 128, 256, 512, 1024), norm=nn.BatchNorm2d, drop=0.1):
    layers = [ResBlock(3, nfs[0], ks=5, stride=1, act=act, norm=norm)]
    layers += [ResBlock(nfs[i], nfs[i+1], act=act, norm=norm, stride=2) 
               for i in range(len(nfs)-1)]
    layers += [upblock(nfs[i], nfs[i-1], act=act, norm=norm) 
               for i in range(len(nfs)-1, 0, -1)]
    layers += [ResBlock(nfs[0], 3, act=nn.Identity, norm=norm)]
    return nn.Sequential(*layers).apply(iw)

In [ ]:
iw = partial(init_weights, leaky=0.1)

In [ ]:
metrics = MetricsCB()
cbs = [DeviceCB(), metrics, ProgressCB(plot=True), MixedPrecision()]
lr_cbs = [DeviceCB(), ProgressCB(), MixedPrecision()]
opt_func = partial(optim.AdamW, eps=1e-5)

In [ ]:
Learner(get_model().apply(iw), dls, F.mse_loss, cbs=lr_cbs, 
        opt_func=opt_func).lr_find(start_lr=1e-4, gamma=1.2)

In [ ]:
epochs = 5
lr = 1e-3
tmax = epochs * len(dls.train)
sched = partial(lr_scheduler.OneCycleLR, max_lr=lr, total_steps=tmax)
xtra = [BatchSchedCB(sched)]
learn = Learner(get_model().apply(iw), dls, F.mse_loss, lr=lr, cbs=cbs+xtra, opt_func=opt_func)

In [ ]:
learn.fit(epochs)

In [ ]:
# predictions, targets and inputs extracted from the model
p, t, inp = learn.capture_preds(inps=True)

In [ ]:
show_images(denorm(inp[:9]), imsize=2)

In [ ]:
show_images(denorm(p[:9]), imsize=2)

These results are quite bad!!

The fundamental problem lies with the AutoEncoder, especially smaller AutoEncoders to carry out the task of super-resolution.

Instead of adding additional complexity, we can instead simplify the model using the UNet architecture.

## **Switching Over To A UNET**

In [ ]:
del(learn)
clean_mem()

The [original paper](https://arxiv.org/abs/1505.04597) which introduced UNets in 2015, for use in medical imaging describes the architecture consisting... 

>.. of a contracting path to capture context and a symmetric expanding path that enables precise localization.

We have referred to the "contracting path" as the **downsampling path** in previous NBs. Conversely, the "symmetric expanding path" is referred to as the **upsampling path**. The original paper used Convolution blocks in both upsampling and downsampling paths, whereas we have used ResNet blocks.

During each stage of the upsampling path, the architecture copies over the activations from the opposite downsampling path. This ensures that the structure of the original image is retained for our super-resolution exercise.

In [ ]:
class TinyUnet(nn.Module):
    def __init__(self, act=act_gr, nfs=(32, 64, 128, 256, 512, 1024), norm=nn.BatchNorm2d):
        super().__init__()
        self.start = ResBlock(3, nfs[0], stride=1, act=act, norm=norm)
        # Downsampling path
        self.dn = nn.ModuleList([ResBlock(nfs[i], nfs[i+1], act=act, norm=norm, stride=2)
                                 for i in range(len(nfs)-1)])
        # Upsampling path
        self.up = nn.ModuleList([upblock(nfs[i], nfs[i-1], act=act, norm=norm)
                                 for i in range(len(nfs)-1, 0, -1)])
        # Final 3 channel block
        self.up += [ResBlock(nfs[0], 3, act=act, norm=norm)]
        self.end = ResBlock(3, 3, act=nn.Identity, norm=norm)

    def forward(self, x):
        # Save first input before starting downsampling. 
        layers = []
        layers.append(x)
        x = self.start(x)
        # For each layer in the downsampling path, save activations.
        for l in self.dn:
            layers.append(x)
            x = l(x)
        n = len(layers)
        # During upsampling, instead of concatenating simply add the saved activations from 
        # the earlier steps
        for i, l in enumerate(self.up):
            if i!=0: x += layers[n-i] # Skip first layer.
            x = l(x)
        # At the end, at add back the first layer and pass through the ResBlock.
        return self.end(x + layers[0])

In [ ]:
# Zero out the weights and biases of the model.
def zero_wgts(l):
    with torch.no_grad():
        l.weight.zero_()
        l.bias.zero_()

In [ ]:
model = TinyUnet()

In [ ]:
last_res = model.up[-1]           # End of the upsampling path
zero_wgts(last_res.convs[-1][-1]) # Zero out the last convolution
zero_wgts(last_res.idconv[0])     # Zero out the last ID connection
zero_wgts(model.end.convs[-1][-1])# Zero out the weights for the end block 

An alternative approach to the above cellblock is that instead of zeroing out the final blocks, we can multiply the weights and activations with a very small number such as `1e-3` to guarantee that there is no signal loss.

In [ ]:
Learner(model, dls, F.mse_loss, cbs=lr_cbs, opt_func=opt_func).lr_find(start_lr=1e-4, gamma=1.2)

In [ ]:
model = TinyUnet()

In [ ]:
last_res = model.up[-1]
zero_wgts(last_res.convs[-1][-1])
zero_wgts(last_res.idconv[0])
zero_wgts(model.end.convs[-1][-1])

In [ ]:
epochs = 20
lr = 1e-2
tmax = epochs * len(dls.train)
sched = partial(lr_scheduler.OneCycleLR, max_lr=lr, total_steps=tmax)
xtra = [BatchSchedCB(sched)]
learn = Learner(model, dls, F.mse_loss, lr=lr, cbs=cbs+xtra, opt_func=opt_func)

In [ ]:
learn.fit(epochs)

In [ ]:
#torch.save(learn.model, 'models/superres-cross')

In [ ]:
p, t, inp = learn.capture_preds(inps=True)

In [ ]:
#Inputs
show_images(denorm(inp[:9]), imsize=2.5)

In [ ]:
# Outputs
show_images(denorm(p[:9]), imsize=2.5)

In [ ]:
# Training samples.
show_images(denorm(t[:9]), imsize=2.5)

Although we haven't recreated the originals, this approach's outputs are a marked improvement over the autoencoder approach.

However, using MSE as a loss function for a model such as this is incorrect and results in blurry outputs like we saw above. This is a result of averaging the loss during the training steps.

To fix this, **Perceptual Loss** is the recommended metric.

## **Perceptual Loss To Replace MSE**

>### **[What Are Perceptual Loss Functions?](https://deepai.org/machine-learning-glossary-and-terms/perceptual-loss-function)**
>
>**Perceptual loss** functions are designed to capture perceptual differences between images, such as content and style discrepancies, which are not always evident at the pixel level. They are often employed in tasks where the goal is to generate images that are visually pleasing to humans, such as in neural style transfer, super-resolution, and image synthesis.
>
>The core idea behind perceptual loss is to use the feature maps from various layers of a CNN, which has been pre-trained on a large dataset like ImageNet. By extracting these feature maps from both the target image and the generated image, we can compute the difference in the high-level features that the network has learned to detect, such as edges, textures, and patterns.
>
>### **How Perceptual Loss Functions Work**
>
>One common approach to implementing perceptual loss involves using a pre-trained VGG network, a type of CNN that has been shown to be effective in capturing image content and style. The perceptual loss function typically consists of two main components:
>
>    **Content Loss**: This measures how much the feature maps of the generated image differ from the feature maps of the target image. By minimizing this loss, the generated image is encouraged to preserve the content of the target image.
>
>    **Style Loss**: This measures the difference in the correlation between feature maps, capturing the texture and style information. Minimizing style loss ensures that the style of the generated image matches the style of a reference image.
>
>During the training process, the generated image is passed through the pre-trained network, and its feature maps are extracted at predetermined layers. The same is done for the target and style reference images. The perceptual loss is then calculated by comparing these feature maps using a distance metric, such as the Euclidean distance.

In [ ]:
# Classifier model
cmodel = torch.load('models/imgnet-tiny-basic-25').cuda()# Train and fill in imgnet-tiny-custom model. For now
                              # lets use the base tiny imgnet model.

In [ ]:
xb, yb = next(iter(dls.valid))
with torch.autocast('cuda'), torch.no_grad(): # Automatic mixed precision
    preds = to_cpu(cmodel(yb.cuda().half()))
preds.shape

In [ ]:
# Using the same code as before for the SynSets
id2str = (path/'wnids.txt').read_text().splitlines()
all_synsets = [o.split('\t') for o in (path/'words.txt').read_text().splitlines()]
synsets = {k:v.split(',', maxsplit=1)[0] for k, v in all_synsets if k in id2str}

In [ ]:
# Titles are off since we are working on model predictions and the model had ~60% accuracy
titles = [synsets[id2str[o]] for o in preds.argmax(dim=1)]
show_images(denorm(yb[:16]), imsize=2.5, titles=titles[:16])

In [ ]:
cmodel

In [ ]:
# Based on the architecture above, we will grab the end of the 3rd Resblock
for i in range(4, len(cmodel)): del(cmodel[4])

In [ ]:
with torch.autocast('cuda'), torch.no_grad():
    feat = to_cpu(cmodel(yb.cuda())).float()
    t = to_cpu(learn.model(yb.cuda())).float()
    pred_feat = to_cpu(cmodel(t.cuda())).float()

feat.shape

The intuitive approach is to ensure that the features in `cmodel` generally have the same sign as features in the `learn.model`.

Moving onto the loss function:

In [ ]:
# Function to calculate MSE Loss b/w inputs and targets, plus the MSE Loss
# for the features we get out of the cmodel and the features of the target image.
def comb_loss(inp, tgt):
    with torch.autocast('cuda'):
        # Target features must not get modified i.e. no_grad()
        with torch.no_grad(): tgt_feat = cmodel(tgt).float() 
        # Inputs with have grad()    
        inp_feat = cmodel(inp).float()
    feat_loss = F.mse_loss(inp_feat, tgt_feat)
    return F.mse_loss(inp, tgt) + feat_loss/10 # standardize feature loss for simplicity.

In [ ]:
def get_unet():
    model = TinyUnet()
    last_res = model.up[-1]
    zero_wgts(last_res.convs[-1][-1])
    zero_wgts(last_res.idconv[0])
    zero_wgts(model.end.convs[-1][-1])
    return model

In [ ]:
Learner(get_unet(), dls, comb_loss, cbs=lr_cbs, opt_func=opt_func).lr_find(start_lr=1e-4, gamma=1.2)

In [ ]:
epochs = 20
lr = 1e-2
tmax = epochs * len(dls.train)
sched = partial(lr_scheduler.OneCycleLR, max_lr=lr, total_steps=tmax)
xtra = [BatchSchedCB(sched)]
learn = Learner(get_unet(), dls, comb_loss, lr=lr, cbs=cbs+xtra, opt_func=opt_func)

In [ ]:
learn.fit(epochs)

In [ ]:
p, t, inp = learn.capture_preds(inps=True)

In [ ]:
show_images(denorm(inp[:9]), imsize=2.5)

In [ ]:
show_images(denorm(p[:9]), imsize=2.5)

In [ ]:
show_images(denorm(t[:9]), imsize=2.5)

## **Unfreezing The Pretrained UNet**

In this section, we will take the pretrained Tiny Imagenet model's weights and copy them over to the ResBlock at the start of the UNet model i.e. `model.start` - effectively fine-tuning the model.

It it important to note that we will have random weights in the upsampling path and pre-trained weights in the downsampling path so setting `requires_grad_(False)` at appropriate stages is key.

`Note` Execute the next few sections once the imgnet custom models have been trained.

In [ ]:
model = get_unet()

In [ ]:
pmodel = torch.load('models/imgnet-tiny-basic-25')

In [ ]:
# All nn.Module components have state_dict() which allows for weights to be
# copied from one model to another.
# Copying ResBlocks ==> pmodel[0] to model.start 
model.start.load_state_dict(pmodel[0].state_dict())
# Since our model has 5 ResBlocks
for i in range(5): model.dn[i].load_state_dict(pmodel[i+1].state_dict())

In [ ]:
for o in model.dn.parameters(): o.requires_grad_(False)

In [ ]:
# Run one epoch of the upsampling path to train the random weights before
# moving onto the downsampling path.
epochs = 1
lr = 3e-3
tmax = epochs * len(dls.train)
sched = partial(lr_scheduler.OneCycleLR, max_lr=lr, total_steps=tmax)
xtra = [BatchSchedCB(sched)]
learn = Learner(model, dls, comb_loss, lr=lr, cbs=cbs+xtra, opt_func=opt_func)

In [ ]:
learn.fit(epochs)

In [ ]:
for o in model.dn.parameters(): o.requires_grad_(True)

In [ ]:
epochs = 20
lr = 3e-3
tmax = epochs * len(dls.train)
sched = partial(lr_scheduler.OneCycleLR, max_lr=lr, total_steps=tmax)
xtra = [BatchSchedCB(sched)]
learn = Learner(model, dls, comb_loss, lr=lr, cbs=cbs+xtra, opt_func=opt_func)

In [ ]:
learn.fit(epochs)

In [ ]:
torch.save(learn.model, 'models/superres-pcp.pkl')
# learn.model = torch.load('models/superres-pcp.pkl').cuda()

In [ ]:
p,t,inp = learn.capture_preds(inps=True)

In [ ]:
show_images(denorm(inp[:9]), imsize=2)

In [ ]:
show_images(denorm(p[:9]), imsize=2)

In [ ]:
show_images(denorm(t[:9]), imsize=2)

## **Cross Connections / Convolutions**

The idea here is to introduce ResBlocks _within_ the cross connections of each downsampling and upsampling layer prior to concatenating or summing the outputs of the layers.

In [ ]:
def cross_conv(nf, act, norm):
    return nn.Sequential(
        ResBlock(nf, nf, act=act, norm=norm),
        nn.Conv2d(nf, nf, 3, padding=1)
    )

In [ ]:
# Rewriting TinyUnet class to incorporate cross connection convs.
class TinyUnet(nn.Module):
    def __init__(self, act=act_gr, nfs=(32, 64, 128, 256, 512, 1024), norm=nn.BatchNorm2d):
        super().__init__()
        self.start = ResBlock(3, nfs[0], ks=5, stride=1, act=act, norm=norm)
        self.dn = nn.ModuleList([ResBlock(nfs[i], nfs[i+1], act=act, norm=norm, stride=2)
                                 for i in range(len(nfs)-1)])
        # Cross connection conv. / resblock
        self.xs = nn.ModuleList([cross_conv(nfs[i], act=act, norm=norm)
                                 for i in range(len(nfs)-1, 0, -1)])
        self.xs += [cross_conv(nfs[0], act=act, norm=norm)]
        self.up = nn.ModuleList([upblock(nfs[i], nfs[i-1], act=act, norm=norm)
                                 for i in range(len(nfs)-1, 0, -1)])
        self.up += [ResBlock(nfs[0], 3, act=act, norm=norm)]
        self.end = ResBlock(3, 3, act=nn.Identity, norm=norm)

    def forward(self, x):
        layers = []
        layers.append(x)
        x = self.start(x)
        for i, l in enumerate(self.dn):
            layers.append(x)
            x = l(x)
        n = len(layers)
        for i, l in enumerate(self.up):
            if i!=0: x += self.xs[i](layers[n-i]) # Add the cross conv
            x = l(x)
        return self.end(x + layers[0])

In [ ]:
pmodel = torch.load('models/inettiny-custom-25')

In [ ]:
model = get_unet()

In [ ]:
model.start.load_state_dict(pmodel[0].state_dict())
for i in range(5): model.dn[i].load_state_dict(pmodel[i+1].state_dict())
for o in model.dn.parameters(): o.requires_grad_(False)

In [ ]:
epochs = 1
lr = 3e-3
sched = partial(lr_scheduler.OneCycleLR, max_lr=lr, total_steps=tmax)
xtra = [BatchSchedCB(sched)]
learn = Learner(model, dls, comb_loss, lr=lr, cbs=cbs+xtra, opt_func=opt_func)

In [ ]:
learn.fit(epochs)

In [ ]:
for o in model.dn.parameters(): o.requires_grad_(True)

In [ ]:
epochs = 20
lr = 1e-2
tmax = epohcs * len(dls.train)
sched = partial(lr_scheduler.OneCycleLR, max_lr=lr, total_steps=tmax)
xtra = [BatchSchedCB(sched)]
learn = Learner(model, dls, comb_loss, lr=lr, cbs=cbs+xtra, opt_func=opt_func)

In [ ]:
learn.fit(epochs)

In [ ]:
p,t,inp = learn.capture_preds(inps=True)

In [ ]:
show_images(denorm(inp[:9]), imsize=2)

In [ ]:
show_images(denorm(p[:9]), imsize=2)

In [ ]:
show_images(denorm(t[:9]), imsize=2)

In [ ]:
torch.save(learn.model, 'models/superres-cross.pkl')